In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from gurobipy import *
import pickle
from itertools import chain, combinations

import math

In [2]:
df = pd.read_csv("data.csv", sep=";", index_col=0)

In [3]:
int_col =[str(i) for i in range(1,21)]

In [4]:
class Project:
    def __init__(self, num=0, desc=0, a1=0, a2=0, w=[1/2, 1/2], cost=1):
        self.num = num
        self.desc = desc
        self.a1 = a1
        self.a2 = a2    
        self.cost = cost
        self.w = w
                
    def cal_value(self):
        self.value = (self.a1/4)*self.w[0] + (self.a2/4)*self.w[1]
        
    def describe(self):
        print("Project " + self.desc)
        print("a1: ", self.a1)
        print("a2: ", self.a2)
        print("cost: ", self.cost)
        print("w: ", self.w)
        print("value: ", self.value)
        

In [5]:
project_list = []
for i in range(20):
    r = df.iloc[i]
    
    p = Project(num=i+1)
    p.desc = r["Action idea name"]
    p.a1 = float(r["Hiilineutraalisuus carbon neutrality (avg)  "].replace(',', '.'))
    p.a2 = float(r["Varautuminen Resilience (avg)  "].replace(',', '.'))
    p.pos_impact = float(r["Positiiviset vaikutukset (avg)  "].replace(',', '.'))
    p.neg_impact = float(r["Negatiiviset vaikutukset (neg. impact)  (avg)  "].replace(',', '.'))
    p.cal_value()
    
    project_list.append(p)

In [6]:
df2 = pd.read_excel("inter_updated.xlsx", header=None)

In [7]:
score_list = []
for i in range(20):
    print("i: ", i)
    for j in range(i, 19):
        print("j: ",j)
        score = df2.iloc[i,j]
        if type(score) == str:
            score_list.append(score.replace(' ', ''))

i:  0
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  1
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  2
j:  2
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  3
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  4
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  5
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  6
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  7
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  8
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  9
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j

In [8]:
score_dict = {'+':0.33, '++': 0.66, '+++': 1, '+++(+)': (1+1.33)/2,'+/-':0, '-':-0.33, '0/+':(0.33/2) }

In [9]:
score_dict.keys()

dict_keys(['+', '++', '+++', '+++(+)', '+/-', '-', '0/+'])

In [10]:
score_dict.values()

dict_values([0.33, 0.66, 1, 1.165, 0, -0.33, 0.165])

In [11]:
score_df = pd.DataFrame()

In [12]:
score_df["effect"] = score_dict.keys()
score_df["gamma"] = score_dict.values()
score_df.to_csv("effect")

In [15]:
inter_subset = []
inter_effect = []

for i in range(20):
    for j in range(i, 19):
        score = df2.iloc[i,j]
        if type(score) == str:
            score = score.replace(' ', '')
            
            try:
                inter_effect.append(score_dict[score])
                inter_subset.append([i,j+1])
            except:
                pass

In [16]:
for i in range(len(inter_subset)):
    print(inter_subset[i])
    print(inter_effect[i])
    print()


[0, 3]
0.165

[0, 5]
0.165

[0, 11]
0.165

[1, 2]
0.66

[1, 3]
0.33

[1, 4]
0.66

[1, 5]
0

[1, 9]
0.66

[1, 11]
0.66

[1, 12]
1.165

[1, 14]
1

[1, 16]
0.66

[2, 3]
0.33

[2, 4]
0.33

[2, 6]
0.33

[2, 8]
0.33

[2, 19]
0.33

[3, 6]
0.33

[3, 8]
0.33

[3, 17]
0.33

[4, 12]
1

[4, 14]
1

[5, 12]
0.33

[5, 14]
0.66

[6, 7]
0.33

[6, 8]
1

[6, 9]
0.33

[6, 10]
0.33

[6, 12]
0.66

[6, 14]
0.66

[6, 16]
0.33

[7, 8]
1

[7, 9]
1

[7, 12]
0.33

[7, 14]
0.33

[8, 11]
0.33

[8, 14]
0.66

[8, 15]
-0.33

[8, 16]
0.33

[8, 17]
0.33

[8, 19]
0.33

[9, 11]
0.33

[9, 12]
0.33

[9, 14]
0.66

[9, 16]
0.33

[9, 17]
0.33

[9, 19]
0.33

[10, 11]
0.33

[10, 12]
0.33

[10, 13]
1

[10, 14]
0.33

[10, 15]
0.66

[10, 16]
0.33

[10, 17]
0.33

[10, 18]
0.33

[10, 19]
0.33

[11, 12]
1

[11, 13]
1

[11, 14]
0.66

[11, 15]
0.33

[11, 16]
0.33

[11, 17]
0.33

[11, 18]
0.33

[11, 19]
0.66

[12, 13]
1

[12, 14]
1

[12, 15]
1

[12, 18]
0.33

[12, 19]
0.66

[13, 14]
0.33

[13, 15]
1

[13, 19]
0.33

[14, 15]
1

[14, 16]
0

In [17]:
class SolveProblem:
    def __init__(self, budget_const_pct, inter_subset, inter_effect, project_list, beta_list):
        self.budget_const_pct = budget_const_pct
        
        self.m = len(project_list)
        self.project_values = [x.value for x in project_list]
        self.project_costs = [x.cost for x in project_list] 
        self.beta_list = beta_list
        
        self.optimal_portfolios = []
        self.optimal_values = []
        
        self.interaction_effects_dict = {}
        
        self.interaction_subsets = inter_subset
        self.inter_gamma = inter_effect
        
    def run(self):
        
        #A model that generates the first optimal solution
        model = Model("Portfolio opimization - Additive")   

        #Indexes
        projects = range(len(project_list))

        z = []

        for i in projects:
            z.append(model.addVar(vtype=GRB.BINARY, name="z_%d" % (i)))
        model.update()

        #budget constraints
        self.budget = self.budget_const_pct * sum(self.project_costs)

        model.addConstr(quicksum(z[j]*self.project_costs[j] for j in projects) <= self.budget, "Budget Constraint")
        model.update()

        #objective function
        model.setObjective(quicksum(z[j]*self.project_values[j] for j in projects))
        model.modelSense = GRB.MAXIMIZE
        model.update()

        #optimize the model
        model.optimize()

        #Collect optimal decision variables

        sol_dict = {}
        sol_list = model.getVars()
        
        for i in range(self.m):
            sol_dict[sol_list[i].VarName] = sol_list[i].X
        
        self.optimal_portfolios.append(sol_dict)
        self.optimal_values.append(model.objVal)
        
        for beta in self.beta_list:
            
            self.interaction_effects = []

            for gamma in self.inter_gamma:
                print("gamma", gamma)
                S_k = gamma*0.73*beta
                print("S_k", S_k)
                self.interaction_effects.append(S_k)
                
            output = np.zeros((len(self.interaction_subsets), 20))
            for i, ix in enumerate(self.interaction_subsets):
                output[i][ix] = 1
            self.interaction_subsets_binary = output.T
            self.interaction_subsets_size = output.sum(axis=1)
            
            self.interaction_effects_dict[beta] = self.interaction_effects
            
                                    
            model = Model("Portfolio optimization - with interaction")

            J = self.m #rename the number of project for the simplicity
            
            #select interaction subsets that should be included in the model
            interaction_subsets = self.interaction_subsets
            interaction_subsets_binary = self.interaction_subsets_binary
            interaction_subsets_size = self.interaction_subsets_size
            interaction_effects= self.interaction_effects
    
            
            #Indexes for the model
            projects = range(J)
            interactions = range(len(self.interaction_subsets))

            #Decision variables z_j
            z = []
            for i in projects:
                z.append(model.addVar(vtype=GRB.BINARY, name="z_%d" % (i)))
            model.update()

            #Decision variable g_k
            g = []
            for k in interactions:
                g.append(model.addVar(vtype=GRB.BINARY, name="g_%d" % (k)))
            model.update()

            #budget constraints
            self.budget = self.budget_const_pct * sum(self.project_costs)

            model.addConstr(quicksum(z[j]*self.project_costs[j] for j in projects) <= self.budget, "Budget Constraint")
            model.update()

            #interaction constraints left hand side
            for k in interactions:
                model.addConstr(quicksum(z[j]*interaction_subsets_binary[j][k] for j in projects) -
                               interaction_subsets_size[k] + 1 <= J*g[k])
            model.update()
            
            #interaction constraints right hand side
            for k in interactions:
                model.addConstr(J*g[k] <= quicksum(z[j]*interaction_subsets_binary[j][k] for j in projects) -
                               interaction_subsets_size[k] + J)
            model.update()
            
            #objective function
            model.setObjective(quicksum(z[j]*self.project_values[j] for j in projects) + quicksum(g[k]*interaction_effects[k] for k in interactions))

            model.modelSense = GRB.MAXIMIZE
            model.update()

            #optimize the model
            model.optimize()

            sol_dict = {}
            sol_list = model.getVars()
            for i in range(J + len(self.interaction_subsets)):
                sol_dict[sol_list[i].VarName] = sol_list[i].X
                
            self.optimal_portfolios.append(sol_dict)
            self.optimal_values.append(model.objVal)
            print("Beta: ", beta)
        
        
        return self.optimal_portfolios

In [18]:
def make_project(w):
    project_list = []
    for i in range(20):
        r = df.iloc[i]

        p = Project(num=i+1)
        p.desc = r["Action idea name"]
        p.a1 = float(r["Hiilineutraalisuus carbon neutrality (avg)  "].replace(',', '.'))
        p.a2 = float(r["Varautuminen Resilience (avg)  "].replace(',', '.'))
        p.pos_impact = float(r["Positiiviset vaikutukset (avg)  "].replace(',', '.'))
        p.neg_impact = float(r["Negatiiviset vaikutukset (neg. impact)  (avg)  "].replace(',', '.'))
        p.w = w
        p.cal_value()

        project_list.append(p)
    return project_list

In [19]:
p1 = make_project([0.5, 0.5])
p2 = make_project([0.67, 0.33])
p3 = make_project([1,0])

In [20]:
s2 = SolveProblem(budget_const_pct=0.5, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p2, beta_list =  list(np.arange(0, 0.1, 0.01)) + list(np.arange(0.1, 1, 0.1))+list(np.arange(1,10,1)))
s2.run()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-08
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0xd73f89d9
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 7.7764000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 8.33585 7.7764 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.335850000000e+00, best bound 8.335850000000e+00, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x44937ce6
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 7.8180770

Root relaxation: objective 8.655540e+00, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.65554    0   43    7.81808    8.65554  10.7%    


Cutting planes:
  Gomory: 1
  Cover: 32
  RLT: 16

Explored 1 nodes (75 simplex iterations) in 0.28 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 8.70979 8.68339 7.94393 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.709787000000e+00, best bound 8.709787000000e+00, gap 0.0000%
Beta:  0.04
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0
S_k 0.0
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 1.165
S_k 0.042522500000000005
gamma 1
S_k 0.0365
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.0365
gamma 1
S_k 0.0365
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.0365
gamma 0.33
S_k 0.012045
gamma 

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x00491d4c
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 8.1327070

Root relaxation: objective 9.533118e+00, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.53312    0   37    8.13271    9.53312  17.2%     -    0s
H    0     0                       9.0762835    9.

  Cover: 38
  Clique: 3
  RLT: 16

Explored 1 nodes (103 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 9.32061 8.25856 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.320614500000e+00, best bound 9.320614500000e+00, gap 0.0000%
Beta:  0.09
gamma 0.165
S_k 0.012045
gamma 0.165
S_k 0.012045
gamma 0.165
S_k 0.012045
gamma 0.66
S_k 0.04818
gamma 0.33
S_k 0.02409
gamma 0.66
S_k 0.04818
gamma 0
S_k 0.0
gamma 0.66
S_k 0.04818
gamma 0.66
S_k 0.04818
gamma 1.165
S_k 0.08504500000000001
gamma 1
S_k 0.073
gamma 0.66
S_k 0.04818
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 1
S_k 0.073
gamma 1
S_k 0.073
gamma 0.33
S_k 0.02409
gamma 0.66
S_k 0.04818
gamma 0.33
S_k 0.02409
gamma 1
S_k 0.073
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.66
S_k 0.04818
gamm

...

  Objective range  [4e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 32.6305250

Root relaxation: objective 7.162865e+01, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   71.62865    0   35   32.63052   71.62865   120%     -    0s
H    0     0                      53.3257250   71.62865  34.3%     -    0s
     0     0   61.87066    0   62   53.32572   61.87066  16.0%     -    0s
H    0     0                      54.4527000   61.87066  13.6%     -    0s
     0     0   60.99897    0   48   54.45270   60.99897  12.0%     -    0s
     0     0   60.93678    0

Found heuristic solution: objective 49.0701250

Root relaxation: objective 1.145709e+02, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  114.57090    0   35   49.07013  114.57090   133%     -    0s
H    0     0                      84.0660250  114.57090  36.3%     -    0s
     0     0   97.72276    0   37   84.06603   97.72276  16.2%     -    0s
     0     0   95.42452    0   60   84.06603   95.42452  13.5%     -    0s
     0     0   95.33206    0   55   84.06603   95.33206  13.4%     -    0s
     0     0   95.30040    0   67   84.06603   95.30040  13.4%     -    0s
     0     0   89.61282    0   43   84.06603   89.61282  6.60%     -    0s
H    0     0                      86.1858000   89.61282  3.98%     -    0s
     0     0   87.39030    0    2   86.18580   87.39030  1.40%     -    0s
     0     0   86.42927    0   44   8

  Objective range  [5e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 65.5097250

Root relaxation: objective 1.575131e+02, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  157.51315    0   35   65.50973  157.51315   140%     -    0s
H    0     0                     114.8063250  157.51315  37.2%     -    0s
     0     0  133.92579    0   63  114.80633  133.92579  16.7%     -    0s
H    0     0                     117.9189000  133.92579  13.6%     -    0s
     0     0  130.53263    0   67  117.91890  130.53263  10.7%     -    0s
     0     0  122.38626    0


     0     0  200.45540    0   35   81.94933  200.45540   145%     -    0s
H    0     0                     145.5466250  200.45540  37.7%     -    0s
     0     0  170.65889    0   37  145.54663  170.65889  17.3%     -    0s
     0     0  166.69200    0   37  145.54663  166.69200  14.5%     -    0s
     0     0  159.01663    0   49  145.54663  159.01663  9.25%     -    0s
     0     0  156.20555    0   69  145.54663  156.20555  7.32%     -    0s
     0     0  156.00204    0   79  145.54663  156.00204  7.18%     -    0s
     0     0  155.75874    0   69  145.54663  155.75874  7.02%     -    0s
     0     0  155.75874    0   68  145.54663  155.75874  7.02%     -    0s
     0     0  150.46788    0   74  145.54663  150.46788  3.38%     -    0s
H    0     0                     149.6520000  150.46788  0.55%     -    0s
     0     0     cutoff    0       149.65200  149.65200  0.00%     -    0s

Cutting planes:
  Gomory: 19
  Cover: 42
  MIR: 1
  Zero half: 2
  RLT: 20

Explored 1 nodes (361 

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': -0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': -0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': -0.0,
  'z_11': 1.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': 1.0,
  'z_19': -0.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': -0.0,
  'g_8': -0.0,
  'g_9': -0.0,
  'g_10': -0.0,
  'g_11': -0.0,
  'g_12': 1.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': -0.0,
  'g_17': 1.0,
  'g_18': 1.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': -0.0,
  'g_23': 1.0,
  'g_24': 1.0,
  'g_25': 1.0,
  'g_26': 1

In [22]:
def draw_table2(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    d3 = pd.DataFrame(d2)
    d3.columns = [round(x, 2) for x in d3.columns]
    d3 = d3.astype(int)
    
    d_list = []

    for col in d3.columns:
        d_list.append(sum(abs(d3[0]-d3[col])))
        
    d3 = d3.append(pd.Series(d_list , index=d3.columns),ignore_index=True)
    d3= d3.set_index(pd.Index(list(df["Action idea name"]) + ["Number of changed decisions"]))
    return d3

In [30]:
may_portfolio = draw_table2(s2)

In [24]:
uti = pd.read_csv("utilities.csv", sep=";")["[0.67, 0.33]"]
uti = [float(u.replace(",", ".")) for u in list(uti)]

In [27]:
def cal_add_uti(pf):
    return sum(list((uti*pf for uti, pf in zip(uti, pf))))

def cal_inter_effect(pf, inter_subset, inter_effect):   
    
    total_effect=0
    for i in range(len(inter_subset)):
        (p1, p2) = inter_subset[i]
        if ((pf[p1] == 1) & (pf[p2] == 1)):
            total_effect += np.mean(uti)*inter_effect[i]
            #print((p1, p2))
            #print(inter_effect[i])
            
    return total_effect

In [32]:
beta_trans_may = []

for pair in list(combinations(may_portfolio.columns[1:], 2)):
    pf1= may_portfolio[pair[0]]
    pf2= may_portfolio[pair[1]]

    a = cal_add_uti(pf1)
    b = cal_inter_effect(pf1, inter_subset, inter_effect)
    a2 = cal_add_uti(pf2)
    b2 = cal_inter_effect(pf2, inter_subset, inter_effect)
    
    beta_com = (a2-a)/(b-b2)
    if not np.isnan(beta_com):
        beta_trans_may.append(beta_com)

/var/folders/m0/4kgydjmn6590r2xq4gdjzq3m0000gn/T/ipykernel_65387/3378302066.py:12: RuntimeWarning: invalid value encountered in double_scalars
  beta_com = (a2-a)/(b-b2)


In [33]:
beta_list = list(set(beta_trans_may))

In [35]:
s3 = SolveProblem(budget_const_pct=0.5, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p2, beta_list = beta_list)
s3.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0xd73f89d9
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 7.7764000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 8.33585 7.7764 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.335850000000e+00, best bound 8.335850000000e+00, gap 0.0000%
gamma 0.165
S_k 0.004000714781088374
gamma 0.165
S_k 0.004000714781088374
gamma 0.165
S_k 0.004000714781088374
gamma 0.66
S_k 0.01600285912

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xa33b0f82
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 8.8898942

Root relaxation: objective 1.189039e+01, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.89039    0   38    8.88989   11.89039  33.8%     -    0s
H    0     0                      10.6361006   11.

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xf2688c05
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 9.6874609

Root relaxation: objective 1.447426e+01, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.47426    0   38    9.68746   14.47426  49.4%   

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x8f4fee05
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 9.5965585

Root relaxation: objective 1.417279e+01, 63 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.17279    0   38    9.59656   14.17279  47.7%     -    0s
H    0     0                      12.1419754   14.

S_k 0.24063344013671714
gamma 1
S_k 0.3645961214192684
gamma 1
S_k 0.3645961214192684
gamma 1
S_k 0.3645961214192684
gamma 0.33
S_k 0.12031672006835857
gamma 0.66
S_k 0.24063344013671714
gamma 0.33
S_k 0.12031672006835857
gamma 1
S_k 0.3645961214192684
gamma 0.33
S_k 0.12031672006835857
gamma 1
S_k 0.3645961214192684
gamma 0.33
S_k 0.12031672006835857
gamma 0.33
S_k 0.12031672006835857
gamma 0.66
S_k 0.24063344013671714
gamma -0.33
S_k -0.12031672006835857
gamma 0.66
S_k 0.24063344013671714
gamma 0.33
S_k 0.12031672006835857
gamma 0.33
S_k 0.12031672006835857
gamma 1
S_k 0.3645961214192684
gamma 0.33
S_k 0.12031672006835857
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xa9db25e1
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-02, 9

...

gamma 1
S_k 0.008900640966742835
gamma 0.33
S_k 0.0029372115190251357
gamma 0.33
S_k 0.0029372115190251357
gamma 0.66
S_k 0.005874423038050271
gamma -0.33
S_k -0.0029372115190251357
gamma 0.66
S_k 0.005874423038050271
gamma 0.33
S_k 0.0029372115190251357
gamma 0.33
S_k 0.0029372115190251357
gamma 1
S_k 0.008900640966742835
gamma 0.33
S_k 0.0029372115190251357
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x95b6b0ee
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (1

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xb6540d54
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 8.5553738

Root relaxation: objective 1.082658e+01, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   10.82658    0   38    8.55537   10.82658  26.5%   

gamma 0.66
S_k 0.11236152450287669
gamma 1
S_k 0.17024473409526772
gamma 1
S_k 0.17024473409526772
gamma 1
S_k 0.17024473409526772
gamma 0.33
S_k 0.056180762251438345
gamma 0.66
S_k 0.11236152450287669
gamma 0.33
S_k 0.056180762251438345
gamma 1
S_k 0.17024473409526772
gamma 0.33
S_k 0.056180762251438345
gamma 1
S_k 0.17024473409526772
gamma 0.33
S_k 0.056180762251438345
gamma 0.33
S_k 0.056180762251438345
gamma 0.66
S_k 0.11236152450287669
gamma -0.33
S_k -0.056180762251438345
gamma 0.66
S_k 0.11236152450287669
gamma 0.33
S_k 0.056180762251438345
gamma 0.33
S_k 0.056180762251438345
gamma 1
S_k 0.17024473409526772
gamma 0.33
S_k 0.056180762251438345
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xde2d52d5
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  

S_k 0.5295500479034626
gamma 1
S_k 0.5295500479034626
gamma 0.66
S_k 0.3495030316162853
gamma 0.33
S_k 0.17475151580814266
gamma 0.33
S_k 0.17475151580814266
gamma 0.33
S_k 0.17475151580814266
gamma 0.33
S_k 0.17475151580814266
gamma 0.66
S_k 0.3495030316162853
gamma 1
S_k 0.5295500479034626
gamma 1
S_k 0.5295500479034626
gamma 1
S_k 0.5295500479034626
gamma 0.33
S_k 0.17475151580814266
gamma 0.66
S_k 0.3495030316162853
gamma 0.33
S_k 0.17475151580814266
gamma 1
S_k 0.5295500479034626
gamma 0.33
S_k 0.17475151580814266
gamma 1
S_k 0.5295500479034626
gamma 0.33
S_k 0.17475151580814266
gamma 0.33
S_k 0.17475151580814266
gamma 0.66
S_k 0.3495030316162853
gamma -0.33
S_k -0.17475151580814266
gamma 0.66
S_k 0.3495030316162853
gamma 0.33
S_k 0.17475151580814266
gamma 0.33
S_k 0.17475151580814266
gamma 1
S_k 0.5295500479034626
gamma 0.33
S_k 0.17475151580814266
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threa

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': -0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': -0.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': 0.0,
  'z_19': 0.0,
  'g_0': 0.0,
  'g_1': -0.0,
  'g_2': 0.0,
  'g_3': 0.0,
  'g_4': 0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': 0.0,
  'g_8': 0.0,
  'g_9': 0.0,
  'g_10': 0.0,
  'g_11': -0.0,
  'g_12': 1.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': 0.0,
  'g_17': 1.0,
  'g_18': 1.0,
  'g_19': -0.0,
  'g_20': 0.0,
  'g_21': -0.0,
  'g_22': 1.0,
  'g_23': 1.0,
  'g_24': 1.0,
  'g_25': 1.0,
  'g_26': 1.0,
  'g_27': 

In [36]:
def integer(l):
    return [int(x) for x in l]

In [37]:
for i in range(len(beta_list)):
    print(beta_list[i])
    pf = integer(list(s3.optimal_portfolios[1:][i].values()))[:20]
    print(pf)
    
    if beta_list[i]<0.1:
        low = math.floor(beta_list[i] * 100)/100.0
        high = round(low + 0.01,2)
        
        print("with_low " + str(low))
        print(pf == list(may_portfolio[low])[:20])
        print("with_high " + str(high))
        print(pf == list(may_portfolio[high])[:20])
    
    else:
        low = math.floor(beta_list[i] * 10)/10.0
        high = round(low + 0.1, 1)
        
        print("with_low" + str(low))
        print(pf == list(may_portfolio[low])[:20])
        print("with_high" + str(high))
        print(pf == list(may_portfolio[high])[:20])
    
    print()
    print()

0.033214734587699246
[0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
with_low 0.03
True
with_high 0.04
False


0.19032979144454143
[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]
with_low0.1
False
with_high0.2
True


0.1863009468327092
[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]
with_low0.1
False
with_high0.2
True


0.31707654448761874
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
with_low0.3
False
with_high0.4
True


0.37261003377274227
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
with_low0.3
False
with_high0.4
True


0.3026306280263046
[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]
with_low0.3
True
with_high0.4
False


0.4418977857371066
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
with_low0.4
True
with_high0.5
True


0.4994467416702307
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
with_low0.4
True
with_high0.5
True


0.5266326226575345
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1,

In [39]:
s4 = SolveProblem(budget_const_pct=0.5,inter_subset=inter_subset, inter_effect=inter_effect, project_list=p2, beta_list = [1000, 10000, 10000000])
s4.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0xd73f89d9
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 7.7764000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 8.33585 7.7764 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.335850000000e+00, best bound 8.335850000000e+00, gap 0.0000%
gamma 0.165
S_k 120.45
gamma 0.165
S_k 120.45
gamma 0.165
S_k 120.45
gamma 0.66
S_k 481.8
gamma 0.33
S_k 240.9
gamma 0.66
S_k 481.8
gamma 0


Solution count 4: 158672 153709 82206 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.586723530500e+05, best bound 1.586723530500e+05, gap 0.0000%
Beta:  10000
gamma 0.165
S_k 1204500.0
gamma 0.165
S_k 1204500.0
gamma 0.165
S_k 1204500.0
gamma 0.66
S_k 4818000.0
gamma 0.33
S_k 2409000.0
gamma 0.66
S_k 4818000.0
gamma 0
S_k 0.0
gamma 0.66
S_k 4818000.0
gamma 0.66
S_k 4818000.0
gamma 1.165
S_k 8504500.0
gamma 1
S_k 7300000.0
gamma 0.66
S_k 4818000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 1
S_k 7300000.0
gamma 1
S_k 7300000.0
gamma 0.33
S_k 2409000.0
gamma 0.66
S_k 4818000.0
gamma 0.33
S_k 2409000.0
gamma 1
S_k 7300000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.66
S_k 4818000.0
gamma 0.66
S_k 4818000.0
gamma 0.33
S_k 2409000.0
gamma 1
S_k 7300000.0
gamma 1
S_k 7300000.0
gamma 0.33
S_k

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': 1.0,
  'z_2': 0.0,
  'z_3': 0.0,
  'z_4': 1.0,
  'z_5': 0.0,
  'z_6': 0.0,
  'z_7': 0.0,
  'z_8': 0.0,
  'z_9': 1.0,
  'z_10': 1.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': 1.0,
  'z_14': 1.0,
  'z_15': 1.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 0.0,
  'z_19': 1.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': 0.0,
  'g_3': 0.0,
  'g_4': 0.0,
  'g_5': 1.0,
  'g_6': -0.0,
  'g_7': 1.0,
  'g_8': 1.0,
  'g_9': 1.0,
  'g_10': 1.0,
  'g_11': 0.0,
  'g_12': 0.0,
  'g_13': 0.0,
  'g_14': 0.0,
  'g_15': 0.0,
  'g_16': 0.0,
  'g_17': 0.0,
  'g_18': 0.0,
  'g_19': 0.0,
  'g_20': 1.0,
  'g_21': 1.0,
  'g_22': -0.0,
  'g_23': 0.0,
  'g_24': -0.0,
  'g_25': 0.0,
  'g_26': 0.0,
  'g_27': 0.0,
  '

In [40]:
s4.optimal_portfolios

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': 1.0,
  'z_2': 0.0,
  'z_3': 0.0,
  'z_4': 1.0,
  'z_5': 0.0,
  'z_6': 0.0,
  'z_7': 0.0,
  'z_8': 0.0,
  'z_9': 1.0,
  'z_10': 1.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': 1.0,
  'z_14': 1.0,
  'z_15': 1.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 0.0,
  'z_19': 1.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': 0.0,
  'g_3': 0.0,
  'g_4': 0.0,
  'g_5': 1.0,
  'g_6': -0.0,
  'g_7': 1.0,
  'g_8': 1.0,
  'g_9': 1.0,
  'g_10': 1.0,
  'g_11': 0.0,
  'g_12': 0.0,
  'g_13': 0.0,
  'g_14': 0.0,
  'g_15': 0.0,
  'g_16': 0.0,
  'g_17': 0.0,
  'g_18': 0.0,
  'g_19': 0.0,
  'g_20': 1.0,
  'g_21': 1.0,
  'g_22': -0.0,
  'g_23': 0.0,
  'g_24': -0.0,
  'g_25': 0.0,
  'g_26': 0.0,
  'g_27': 0.0,
  '

In [44]:
p10 = p2

In [46]:
for project in p10:
    project.value = 0

In [47]:
p10[0].value

0

In [48]:
s5 = SolveProblem(budget_const_pct=0.5,inter_subset=inter_subset, inter_effect=inter_effect, project_list=p10, beta_list = [1000, 10000, 10000000])
s5.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x2204325f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
gamma 0.165
S_k 120.45
gamma 0.165
S_k 120.45
gamma 0.165
S_k 120.45
gamma 0.66
S_k 481.8
gamma 0.33
S_k 240.9
gamma 0.66
S_k 481.8
gamma 0
S_k 0.0
gamma 0.66
S_k 481.8
gamma 0.66
S_k 481.8
gamma 1.165
S_k 850.45


gamma 0.33
S_k 2409000.0
gamma 1
S_k 7300000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 0.66
S_k 4818000.0
gamma -0.33
S_k -2409000.0
gamma 0.66
S_k 4818000.0
gamma 0.33
S_k 2409000.0
gamma 0.33
S_k 2409000.0
gamma 1
S_k 7300000.0
gamma 0.33
S_k 2409000.0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x669018f5
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+06, 9e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)

Root relaxation: objective 2.147112e+08, 133 iterations, 0.00 seconds (0.00 wo

[{'z_0': -0.0,
  'z_1': -0.0,
  'z_2': -0.0,
  'z_3': -0.0,
  'z_4': -0.0,
  'z_5': -0.0,
  'z_6': -0.0,
  'z_7': -0.0,
  'z_8': -0.0,
  'z_9': -0.0,
  'z_10': -0.0,
  'z_11': -0.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': -0.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': -0.0,
  'z_19': -0.0},
 {'z_0': -0.0,
  'z_1': 1.0,
  'z_2': -0.0,
  'z_3': -0.0,
  'z_4': 1.0,
  'z_5': -0.0,
  'z_6': 0.0,
  'z_7': -0.0,
  'z_8': -0.0,
  'z_9': 1.0,
  'z_10': 1.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': 1.0,
  'z_14': 1.0,
  'z_15': 1.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': -0.0,
  'z_19': 1.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': 1.0,
  'g_6': -0.0,
  'g_7': 1.0,
  'g_8': 1.0,
  'g_9': 1.0,
  'g_10': 1.0,
  'g_11': -0.0,
  'g_12': -0.0,
  'g_13': -0.0,
  'g_14': -0.0,
  'g_15': -0.0,
  'g_16': -0.0,
  'g_17': -0.0,
  'g_18': -0.0,
  'g_19': -0.0,
  'g_20': 1.0,
  'g_21': 1.0,
  'g_22': -0.0,
  'g_23': -0.0,
  'g_24': -0.0,
  'g_25

In [49]:
s5.optimal_portfolios

[{'z_0': -0.0,
  'z_1': -0.0,
  'z_2': -0.0,
  'z_3': -0.0,
  'z_4': -0.0,
  'z_5': -0.0,
  'z_6': -0.0,
  'z_7': -0.0,
  'z_8': -0.0,
  'z_9': -0.0,
  'z_10': -0.0,
  'z_11': -0.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': -0.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': -0.0,
  'z_19': -0.0},
 {'z_0': -0.0,
  'z_1': 1.0,
  'z_2': -0.0,
  'z_3': -0.0,
  'z_4': 1.0,
  'z_5': -0.0,
  'z_6': 0.0,
  'z_7': -0.0,
  'z_8': -0.0,
  'z_9': 1.0,
  'z_10': 1.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': 1.0,
  'z_14': 1.0,
  'z_15': 1.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': -0.0,
  'z_19': 1.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': 1.0,
  'g_6': -0.0,
  'g_7': 1.0,
  'g_8': 1.0,
  'g_9': 1.0,
  'g_10': 1.0,
  'g_11': -0.0,
  'g_12': -0.0,
  'g_13': -0.0,
  'g_14': -0.0,
  'g_15': -0.0,
  'g_16': -0.0,
  'g_17': -0.0,
  'g_18': -0.0,
  'g_19': -0.0,
  'g_20': 1.0,
  'g_21': 1.0,
  'g_22': -0.0,
  'g_23': -0.0,
  'g_24': -0.0,
  'g_25

In [142]:
draw_table2(s2).to_csv("67_33_50.csv", sep=";") #[0.67, 0.33], 0.5
draw_table2(s1).to_csv("50_50_50.csv", sep=";") #[0.5, 0.5], 0.5
draw_table2(s3).to_csv("100_0_50.csv", sep=";") #[1,0], 0.5
draw_table2(s5).to_csv("67_33_25.csv", sep=";") #[0.67, 0.33], 0.25
draw_table2(s6).to_csv("67_33_75.csv", sep=";") #[0.67, 0.33], 0.75


In [176]:
draw_table2(s6)

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.90,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00
"(1) Promoting the energy efficiency of buildings, the use of waste heat and the use and production of renewable energy",1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(2) Enabling decentralized energy production from a security of supply perspective,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
(3) Development and deployment of non-combustion energy technologies and energy storage,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,0,0,0,0
"(4) Promoting the production of renewable energy, such as wind, geo-energy and solar energy",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
(5) Improving energy self-sufficiency to prepare for energy supply disruptions,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
(6) Oil and peat are used only as maintenance security fuels in energy production,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
(7) Improving the climate resilience of arable farming and forests,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"(8) Ensuring the sustainable production, use and renewal of biomass",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
(9) Climate management and use of forests. The growth condition of forests is taken care of and the development of forest carbon balances is monitored,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
(10) Promoting the development of manure treatment methods and the use of manure for biogas and fertilizer production,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
import matplotlib
matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]


In [ ]:
draw_graph(s1)

In [ ]:
df = draw_table(s2)

In [ ]:
df.style.applymap(lambda x: "background-color: green" if x>0 else "background-color: white")

In [ ]:
draw_graph(s2)

In [ ]:
dict_values = []

for pf in s2.optimal_portfolios:
    dict_values.append(list(pf.values())[:20])
d2 = dict(zip(s2.interaction_effects_dict.keys(), dict_values[1:]))

In [ ]:
np.array(d2[0]) - np.array(d2[9])

In [ ]:
def draw_table(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    d3 = pd.DataFrame(d2)
    d3.columns = [round(x, 2) for x in d3.columns]
    d3 = d3.astype(int)
    d3 = d3.set_index(pd.Index(list(range(1,21))))
    return d3

In [ ]:
draw_table(s2)

In [ ]:
def draw_graph(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    
    diff_dict = {}
    for k,v in d2.items():
        diff_dict[k] = np.sum(abs(np.array(d2[0])-np.array(v)))
    plt.plot(diff_dict.keys(), diff_dict.values())
    plt.xlabel(r"$\beta$")
    
    plt.ylabel("")

In [ ]:
draw_graph(s2)

In [ ]:
draw_graph(s1)

In [ ]:
draw_graph(s3)

In [ ]:
p1[0].describe()

In [ ]:
p2[0].describe()

In [ ]:
p3[0].describe()

In [ ]:
draw_graph(s4)

In [ ]:
draw_graph(s5)

In [ ]:
draw_graph(s6)

In [ ]:
dict_values = []

for pf in p.optimal_portfolios:
    dict_values.append(list(pf.values())[:20])

In [ ]:
d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))

In [ ]:
d3 = pd.DataFrame(d2)
d3.columns = [round(x, 1) for x in d3.columns]
d3 = d3.astype(int)

In [ ]:
d3.style.applymap(lambda x: "background-color: blue" if x >0 else 'background-color: white')

In [ ]:
diff_dict = {}
for k,v in d2.items():
    diff_dict[k] = np.sum(abs(np.array(d2[0])-np.array(v)))

In [ ]:
plt.plot(diff_dict.keys(), diff_dict.values())
plt.xlabel("$\beta$")
plt.ylabel("Number of different projects selected")

In [ ]:
dict_values = []

for pf in p.optimal_portfolios:
    dict_values.append(list(pf.values())[20:])
    
d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
d3 = pd.DataFrame(d2)
d3.columns = [round(x, 1) for x in d3.columns]
d3 = d3.astype(int)
d3.style.applymap(lambda x: "background-color: blue" if x >0 else 'background-color: white')

In [ ]:
np.sum(abs(np.array(d2[0])-np.array(d2[0.1])))

In [ ]:
len(p.interaction_effects_dict.keys())

In [ ]:
len(p.optimal_portfolios[1:])

In [ ]:
list(p.optimal_portfolios[13].values())[:20]

In [ ]:
np.dot([1,2,3], [0,1,0])

In [ ]:
p.optimal_portfolios[2]

In [ ]:
x = np.dot(list(p.optimal_portfolios[3].values())[:20], p.project_values)
y = np.dot(list(p.optimal_portfolios[3].values())[20:], p.interaction_effects_dict[0.2])
x+y

In [ ]:
p.optimal_values